# Dark Souls II Reviews (2025)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import altair as alt
import re

In [2]:
df = pd.read_csv('reviews.csv')
reviews = df.copy()
reviews = reviews.set_index('recommendationid')
reviews.drop(columns={'Unnamed: 0', 'in_early_access'}, inplace=True)

Converting date of review from unix:

In [3]:
reviews['month_name'] = pd.to_datetime(reviews.update_date, unit='s').dt.month_name()
reviews['month']      = pd.to_datetime(reviews.update_date, unit='s').dt.month
reviews['year']       = pd.to_datetime(reviews.update_date, unit='s').dt.year
reviews['day']        = pd.to_datetime(reviews.update_date, unit='s').dt.day

Focusing on just the English reviews:

In [4]:
reviews = reviews[reviews.language == 'english']

## Cleaning up the reviews

In [5]:
reviews = reviews.dropna(subset=['review'])
reviews.shape

(45563, 9)

In [6]:
reviews['review'] = reviews.review.str.lower()

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = word_tokenize(text)
    return ' '.join([word for word in words if word.lower() not in stop_words])

In [8]:
reviews['review'] = reviews.review.apply(remove_stopwords)

In [9]:
# Removing urls:
r = [re.sub(r'http\S+', '', review).lower().strip() if pd.notna(review) else review for review in reviews.review]

# Removing esc sequences, punctuation, and numbers:
    # There's some ASCII art in some of the reviews
r = [re.sub(r'[^a-z]', ' ', review).strip() if pd.notna(review) else review for review in r]

In [10]:
# Removing multiple and trailing whitespaces:
r = [re.sub(r' +', ' ', review).strip() if pd.notna(review) else review for review in r]

In [11]:
reviews['review'] = r

## Looking at reviews after DLC release:

In [12]:
dlc = reviews[reviews.review.str.contains(r'\bdlc\b')]
dlc

,review,language,init_date,update_date,voted_up,month_name,month,year,day
recommendationid,,,,,,,,,
190389625,great entry souls trilogy game bit slower othe...,english,1742149455,1742149455,True,March,3,2025,16
190375891,good game dlc fucking shit spamming every fuck...,english,1742140735,1742140735,True,March,3,2025,16
190333872,game fucking sucks ass still much fun play idk...,english,1742107173,1742107197,True,March,3,2025,16
190253620,messed pc controls game beginning linear gets ...,english,1742042235,1742042235,True,March,3,2025,15
190070762,thinking starting dsii unsure version choose l...,english,1741847010,1741847010,True,March,3,2025,13
...,...,...,...,...,...,...,...,...,...
15163660,amazing game well worth price full game upgrad...,english,1427941752,1449432156,True,December,12,2015,6
15163646,s pretty good much changed n t bought original...,english,1427941599,1427941599,True,April,4,2015,2
15162890,ok honestly feel upgraded graphics fixed enimi...,english,1427936509,1427939907,True,April,4,2015,2


- 1st DLC launched: 7/22/2014
- 2nd DLC launched: 8/26/2014
- 3rd DLC launched: 9/30/2014
- Scholar of the First Sin Edition launched: 4/1/2015

In [13]:
dlc_reviews = reviews[(reviews.year >= 2014) & (reviews.month >= 7) & (reviews.day >= 22)]
dlc_reviews = dlc_reviews[dlc_reviews.review.str.contains(r'\bdlc\b')]
dlc_reviews

,review,language,init_date,update_date,voted_up,month_name,month,year,day
recommendationid,,,,,,,,,
184244595,like even though people kind hate game somewha...,english,1735584348,1735584348,True,December,12,2024,30
184165897,started game trying think negatives heard yet ...,english,1735515416,1735515416,False,December,12,2024,29
183613472,unironically masterpiece favourite dark souls ...,english,1735091925,1735091925,True,December,12,2024,25
183467284,game really really overhated although scholar ...,english,1734952064,1734952064,True,December,12,2024,23
183432730,genuinely n t understand amount hate game gets...,english,1734911344,1734911344,True,December,12,2024,22
...,...,...,...,...,...,...,...,...,...
15664976,hollow good buy n t already game dlc ve played...,english,1430634124,1440388162,True,August,8,2015,24
15470237,m changing review currently reflect current st...,english,1429572560,1448541454,True,November,11,2015,26
15296462,avid crazy player dark souls note first portio...,english,1428698609,1451422989,False,December,12,2015,29


In [14]:
dlc_reviews.voted_up.value_counts(normalize=True)

voted_up
True     0.79564
False    0.20436
Name: proportion, dtype: float64